In [1]:
import pandas as pd

In [25]:
heaton_traffic_data = pd.read_parquet("heaton_traffic.parquet")
heaton_sensor_data = pd.read_parquet("heaton_sensors.parquet")


In [26]:
heaton_traffic_df["veh_class"].value_counts()

veh_class
person     1890512
car        1138521
pc          838072
lgv         613023
psv         514257
buggy       226218
ogv1        159008
mc          128098
scooter      33359
ogv2         12554
Name: count, dtype: int64